In [133]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Fetched 108 

In [95]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-02-14 15:14:34--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.07MB/s    in 0.2s    

2023-02-14 15:14:34 (4.07 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [96]:
# Start a SparkSession
import findspark
findspark.init()

In [97]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MobileCloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

Mobile Reviews

In [107]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://amazonreviews011.s3.eu-west-2.amazonaws.com/amazon_reviews_us_Mobile_Electronics_v1_00.tsv" 
spark.sparkContext.addFile(url)
Mobile_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Mobile_Electronics_v1_00.tsv"), sep="\t", header=True)
# Show DataFrame
Mobile_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

In [108]:
Mobile_df.count()

104975

In [109]:
df_mobile = Mobile_df.dropna()
df_mobile.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

In [110]:
# Final count of records in df_camera
df_mobile.count()

104972

Filtering Reviews by Votes

In [134]:
VineReviews_df = df_mobile.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
VineReviews_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R8MEA6IGAHO0B|          5|            0|          0|   N|                Y|
|R31LOQ8JGLPRLK|          5|            0|          1|   N|                Y|
|R2Y0MM9YE6OP3P|          5|            0|          0|   N|                Y|
| RRB9C05HDOD4O|          4|            0|          0|   N|                Y|
|R26I2RI1GFV8QG|          2|            0|          0|   N|                Y|
| RY8DDL22YG4R5|          3|            0|          1|   N|                Y|
|R2AT2426ZHFUHH|          3|            0|          1|   N|                Y|
|R3RRXU2R23NMQ9|          5|            0|          0|   N|                Y|
|R250PR8VJUZ62F|          4|            0|          2|   N|                Y|
| RBEMQ29WJBHYG|          5|          164|        168|   N|     

In [135]:
VineReviews_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [136]:
from pyspark.sql.types import *
VineReviews_df = VineReviews_df.withColumn("star_rating",VineReviews_df["star_rating"].cast(IntegerType()))\
    .withColumn("helpful_votes",VineReviews_df["helpful_votes"].cast(IntegerType()))\
    .withColumn("total_votes",VineReviews_df["total_votes"].cast(IntegerType()))
VineReviews_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R8MEA6IGAHO0B|          5|            0|          0|   N|                Y|
|R31LOQ8JGLPRLK|          5|            0|          1|   N|                Y|
|R2Y0MM9YE6OP3P|          5|            0|          0|   N|                Y|
| RRB9C05HDOD4O|          4|            0|          0|   N|                Y|
|R26I2RI1GFV8QG|          2|            0|          0|   N|                Y|
| RY8DDL22YG4R5|          3|            0|          1|   N|                Y|
|R2AT2426ZHFUHH|          3|            0|          1|   N|                Y|
|R3RRXU2R23NMQ9|          5|            0|          0|   N|                Y|
|R250PR8VJUZ62F|          4|            0|          2|   N|                Y|
| RBEMQ29WJBHYG|          5|          164|        168|   N|     

In [137]:
VineReviews_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [138]:
df3 = VineReviews_df.filter(VineReviews_df["total_votes"] >= 20)
df3.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RBEMQ29WJBHYG|          5|          164|        168|   N|                Y|
|R2JK5Y8D5MXAGP|          1|            5|        121|   N|                Y|
|R13W3EMIUV120L|          4|          288|        294|   N|                Y|
|R30TJ8POYNVCRE|          5|           15|         21|   N|                N|
|R1H5Y9Z2DHRNVO|          1|            4|         95|   N|                Y|
|R1LK03P7BCJEJD|          1|           31|         35|   N|                Y|
| RBXUICKGHL685|          1|           48|         52|   N|                N|
|R2E16WLZKP78GO|          5|           19|         21|   N|                Y|
|R1EN4PGQIFSENW|          5|           27|         28|   N|                Y|
| RXZWQU66AHIYG|          5|           24|         25|   N|     

In [139]:
df4 = df3.filter(df3["helpful_votes"]/df3["total_votes"] >= 0.5)
df4.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RBEMQ29WJBHYG|          5|          164|        168|   N|                Y|
|R13W3EMIUV120L|          4|          288|        294|   N|                Y|
|R30TJ8POYNVCRE|          5|           15|         21|   N|                N|
|R1LK03P7BCJEJD|          1|           31|         35|   N|                Y|
| RBXUICKGHL685|          1|           48|         52|   N|                N|
|R2E16WLZKP78GO|          5|           19|         21|   N|                Y|
|R1EN4PGQIFSENW|          5|           27|         28|   N|                Y|
| RXZWQU66AHIYG|          5|           24|         25|   N|                Y|
|R2IXCZCSUKH8FB|          3|           78|         82|   N|                Y|
|R25RB38U9LL8OE|          5|           24|         29|   N|     

Stats on Vine Reviews

In [140]:
from pyspark.sql.functions import col, avg
paid_df = df4.filter(df4['vine']== 'Y')
paid_df.describe().show()

+-------+--------------+-----------------+------------------+------------------+----+-----------------+
|summary|     review_id|      star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+--------------+-----------------+------------------+------------------+----+-----------------+
|  count|             4|                4|                 4|                 4|   4|                4|
|   mean|          null|              4.0|            177.75|             205.5|null|             null|
| stddev|          null|0.816496580927726|174.85684621045488|193.88226668092503|null|             null|
|    min|R1D6REC9HPJVQY|                3|                31|                41|   Y|                N|
|    max| RWF03LXVXC22A|                5|               396|               445|   Y|                N|
+-------+--------------+-----------------+------------------+------------------+----+-----------------+



In [141]:
unpaid_df = df4.filter(df4['vine']== 'N')
unpaid_df.describe().show()

+-------+--------------+------------------+------------------+------------------+----+-----------------+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+--------------+------------------+------------------+------------------+----+-----------------+
|  count|          1064|              1064|              1064|              1064|1064|             1064|
|   mean|          null| 3.732142857142857|43.431390977443606|  47.3656015037594|null|             null|
| stddev|          null|1.5550907855049756|50.444002200341956|52.351090444777974|null|             null|
|    min|R104R6XNOY3EL1|                 1|                11|                20|   N|                N|
|    max| RZOAB4T6LBDE8|                 5|               769|               791|   N|                Y|
+-------+--------------+------------------+------------------+------------------+----+-----------------+



Percentage 5 STAR Reviews by Vine (paid) customers

In [142]:
paid_five_star_number = paid_df[paid_df['star_rating']== 5].count()
paid_number = paid_df.count()
percentage_five_star_vine = float(paid_five_star_number) / float(paid_number)
print(paid_number)
print(paid_five_star_number)
print(percentage_five_star_vine)

4
1
0.25


Pencentage 5 STAR Reviews by normal customers (Vine = N)

In [143]:
unpaid_five_star_number = unpaid_df[unpaid_df['star_rating']== 5].count()
unpaid_number = unpaid_df.count()
percentage_five_star_non_vine = float(unpaid_five_star_number) / float(unpaid_number)
print(unpaid_number)
print(unpaid_five_star_number)
print(percentage_five_star_non_vine)

1064
527
0.49530075187969924


In [148]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cmmggpkmm8qx.eu-west-2.rds.amazonaws.com:5432/AmazonReview_Mobile"
config = {"user":"root", 
          "password": "password1", 
          "driver":"org.postgresql.Driver"}

In [149]:
VineReviews_df.write.jdbc(url=jdbc_url, table="vine_reviews_mobile", mode=mode, properties=config)